In [1]:
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.sql_database import SQLDatabase
from langchain_experimental.sql import SQLDatabaseChain
from langchain.agents import initialize_agent, Tool
from langchain.agents import load_tools
from langchain.memory import ConversationBufferMemory

from rw_formula import RW_Calc2

In [2]:
import os
from dotenv import load_dotenv
load_dotenv()
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

In [3]:
llm = OpenAI(temperature=0)
# llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo-0613")
db = SQLDatabase.from_uri("sqlite:///./rwa_data.db")
db_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True)
tools = [
    Tool(
        name="risk weight calculator",
        func=RW_Calc2,
        description=
        """calculates a loan risk weight give the argulamts: 
        Segment - Possible values are 'Bank', 'Corporate', and 'Retail'), 
        PD - Probability of Default
        LGD - Loss Given Default
        m - Remaining maturity of the loan in years
        Large_Fin - If 'Y' the client is a Flag for Large Financial Institution, otherwise 'N'
        size - size of the client in MEUR, usually this is the client's turnover
        mortgage - If 'Y' the exposure is a mortgage loan, otherwise 'N'
        revolving - If 'Y' the exposure is a revolving loan, otherwise 'N'
        """
    ),     
    Tool(
        name="rwa_db",
        func=db_chain.run,
        description="useful for quering a database"
    )
]

In [4]:
memory = ConversationBufferMemory(memory_key="chat_history")

In [5]:
agent = initialize_agent(
    agent='conversational-react-description', 
    tools=tools, 
    llm=llm,
    verbose=True,
    max_iterations=5,
    memory=memory,
    handle_parsing_errors=True
)

In [6]:
agent.run("Report the total RWA and EAD per SEGMENT from RWA_122022 table")



> Entering new AgentExecutor chain...

Thought: Do I need to use a tool? Yes
Action: rwa_db
Action Input: SELECT SUM(RWA) AS RWA, SUM(EAD) AS EAD FROM RWA_122022 GROUP BY SEGMENT

> Entering new SQLDatabaseChain chain...
SELECT SUM(RWA) AS RWA, SUM(EAD) AS EAD FROM RWA_122022 GROUP BY SEGMENT
SQLQuery:SELECT SUM(RWA) AS RWA, SUM(EAD) AS EAD FROM "RWA_122022" GROUP BY "SEGMENT" LIMIT 5;
SQLResult: [(85075812.12498187, 156657332.34921697), (37806680.90001285, 165698955.81288758)]
Answer:The total RWA and EAD for the RWA_122022 table grouped by segment are 85075812.12498187 and 156657332.34921697 respectively.
> Finished chain.

Observation: The total RWA and EAD for the RWA_122022 table grouped by segment are 85075812.12498187 and 156657332.34921697 respectively.
Thought: Do I need to use a tool? No
AI: Is there anything else I can help you with?

> Finished chain.


'Is there anything else I can help you with?'

In [7]:
agent.run("Read the information for Client 'sfman997ux8036' in RWA_122022 table and then calculate the RW if PD doubles")



> Entering new AgentExecutor chain...

Thought: Do I need to use a tool? Yes
Action: rwa_db
Action Input: Read the information for Client 'sfman997ux8036' in RWA_122022 table

> Entering new SQLDatabaseChain chain...
Read the information for Client 'sfman997ux8036' in RWA_122022 table
SQLQuery:SELECT "CLIENT_NUMBER", "ACCOUNT_NUMBER", "PRODUCT", "STATUS", "SEGMENT", "EXPOSURE_DRAWN", "EXPOSURE_UNDRAWN", "EXPOSURE", "EAD", "EAD_COLL", "PD", "LGD", "SIZE", "MATURITY", "F_MORTGAGE", "F_REVOLVING", "F_LARGE_FIN", "RW", "RWA" FROM "RWA_122022" WHERE "CLIENT_NUMBER" = 'sfman997ux8036' LIMIT 5;
SQLResult: [('sfman997ux8036', '1324710203qiph20', 'CREDIT_CARD', 'Performing', 'Retail', 0.0, 18728.13876, 18728.13876, 11004.65713, 0.0, 0.0246518, 0.3092, 42.58029755, 1.15616438, 'N', 'Y', 'N', 0.245013691, 2696.291657)]
Answer:The information for Client 'sfman997ux8036' in RWA_122022 table is: Client Number: sfman997ux8036, Account Number: 1324710203qiph20, Product: CREDIT_CARD, Status: Performi

'The risk weight for the given parameters is 0.3950618648985617. Is there anything else I can help you with?'

In [8]:
agent.run("What is the sum of all EXPOSURE in RWA_122022 for all ACCOUNT_NUMBER whose STATUS changed from Performing to Non Performing between RWA_122022 table and RWA_032023 table")



> Entering new AgentExecutor chain...

Thought: Do I need to use a tool? Yes
Action: rwa_db
Action Input: Sum of all EXPOSURE in RWA_122022 for all ACCOUNT_NUMBER whose STATUS changed from Performing to Non Performing between RWA_122022 table and RWA_032023 table

> Entering new SQLDatabaseChain chain...
Sum of all EXPOSURE in RWA_122022 for all ACCOUNT_NUMBER whose STATUS changed from Performing to Non Performing between RWA_122022 table and RWA_032023 table
SQLQuery:SELECT SUM(RWA_122022.EXPOSURE) 
FROM RWA_122022 
INNER JOIN RWA_032023 
ON RWA_122022.ACCOUNT_NUMBER = RWA_032023.ACCOUNT_NUMBER 
WHERE RWA_122022.STATUS = 'Performing' 
AND RWA_032023.STATUS = 'Non Performing'
SQLResult: [(575151.3173016,)]
Answer:The sum of all EXPOSURE in RWA_122022 for all ACCOUNT_NUMBER whose STATUS changed from Performing to Non Performing between RWA_122022 table and RWA_032023 table is 575151.3173016.
> Finished chain.

Observation: The sum of all EXPOSURE in RWA_122022 for all ACCOUNT_NUMBER w

'Is there anything else I can help you with?'